In [ ]:
!pip install kiwipiepy pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 24.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 14.2 MB/s eta 0:00:00
  Created wheel for kiwipiepy_model: filename=kiwipiepy_model-0.20.0-py3-none-any.whl size=34818026 sha256=218817eb3878924669acc91d7088ddd73cd1f6e9c938967564ba0239be81d0f0
  Stored in directory: /root/.cache/pip/wheels/ca/c8/52/3a539d6e9065b191fe1c215e0203dcc3e00601c0e3d3d39824
Successfully built kiwipiepy_model


In [ ]:
import pandas as pd
from kiwipiepy import Kiwi
import kiwipiepy
import os

# 형태소 분석하기

In [ ]:
import pandas as pd
from kiwipiepy import Kiwi
import os

# 1. CSV 파일 경로 설정 (실제 파일 경로로 수정하세요)
merged_df_file = '/content/2025-01-21-12_final_lower.csv'          # 입력 CSV 파일 경로
user_dictionary_file = '/content/user_dict_lower.txt'   # 사용자 사전 파일 경로
output_file = '/content/[lower]analyzed_job_data_final.csv'            # 출력 CSV 파일 경로

# 2. CSV 파일 읽기
try:
    merged_df = pd.read_csv(merged_df_file)
    print("CSV 파일 읽기 완료.")
except FileNotFoundError:
    print(f"오류: 파일을 찾을 수 없습니다. 경로를 확인하세요: {merged_df_file}")
    exit(1)

# 3. Kiwi 초기화 및 사용자 사전 적용
# kiwi = Kiwi(typos='basic_with_continual_and_lengthening')  # 기본 오타 정보, 연철, 장음화 함께 사용
kiwi = Kiwi(typos='basic_with_continual')

# 3.1. 사용자 사전 파일에서 단어 읽기
try:
    with open(user_dictionary_file, 'r', encoding='utf-8') as f:
        user_custom_words = [line.strip() for line in f if line.strip()]
    print(f"사용자 사전 단어 수: {len(user_custom_words)}개")
except FileNotFoundError:
    print(f"오류: 사용자 사전 파일을 찾을 수 없습니다. 경로를 확인하세요: {user_dictionary_file}")
    exit(1)

# 3.2. 사용자 사전 단어를 NNP 품사로 등록
for word in user_custom_words:
    kiwi.add_user_word(word, 'SL')  # IT 용어는 보통 고유 명사(NNP)로 분류

print("사용자 사전 단어 등록 완료.")

CSV 파일 읽기 완료.
사용자 사전 단어 수: 232개
사용자 사전 단어 등록 완료.


In [ ]:
# 5. 형태소 분석 함수 정의
allowed_pos_tags = ['SL']  # 외래어만 추출

def analyze_foreign_words(text):
    if not isinstance(text, str):
        return []
    # 형태소 분석 수행 (불용어 제거 및 IT 용어 고정)
    tokens = kiwi.tokenize(text, normalize_coda=True, split_complex=True)
    # 외래어(SL)만 추출
    foreign_words = [morph for morph, pos, _, _ in tokens if pos == 'SL']
    # 중복된 토큰 제거 (순서 유지)
    seen = set()
    unique_foreign_words = []
    for morph in foreign_words:
        if morph not in seen:
            seen.add(morph)
            unique_foreign_words.append(morph)
    return unique_foreign_words

def analyze_foreign_words_all(text):
    if not isinstance(text, str):
        return []
    # 형태소 분석 수행 (불용어 제거 및 IT 용어 고정)
    tokens = kiwi.tokenize(text, normalize_coda=True, split_complex=True)
    combined_tokens = []
    morph_seen = set()
    i = 0
    while i < len(tokens):
        morph, pos, _, _ = tokens[i]
        # 현재 토큰이 SN이고 다음 토큰이 NNB인 경우 결합 (외래어 포함)
        if pos == 'SN' and (i + 1) < len(tokens):
            next_morph, next_pos, _, _ = tokens[i + 1]
            if next_pos == 'NNB':
                combined_morph = morph + next_morph
                combined_pos = 'SN+NNB'
                if pos == 'SL' and combined_morph not in morph_seen:
                    combined_tokens.append(f"{combined_morph}+{combined_pos}")
                    morph_seen.add(combined_morph)
                i += 2
                continue
        # 외래어(SL)만 처리
        if pos == 'SL' and morph not in morph_seen:
            combined_tokens.append(f"{morph}+{pos}")
            morph_seen.add(morph)
        i += 1
    return combined_tokens


In [ ]:

# 6. 필요한 열이 존재하는지 확인하고, 없으면 빈 문자열로 채움
required_columns = ['description', 'requirement', 'preferredExperience']
for col in required_columns:
    if col not in merged_df.columns:
        merged_df[col] = ''

# 7. 'description'과 'requirement'를 합쳐 새로운 열 생성
merged_df['description_requirement'] = (
    merged_df['description'].fillna('') + ' ' +
    merged_df['requirement'].fillna('')
)

# 8. 'preferredExperience'는 별도로 새로운 열 생성 (필요시 다른 전처리 추가 가능)
merged_df['preferredExperience_cleaned'] = merged_df['preferredExperience'].fillna('')

# 9. 형태소 분석 적용
print("형태소 분석 시작...")

# 분석 함수 호출 시 사용될 함수명 수정
merged_df['combined_analyzed'] = merged_df['description_requirement'].apply(analyze_foreign_words)
merged_df['combined_analyzed_all'] = merged_df['description_requirement'].apply(analyze_foreign_words_all)
merged_df['preferredExperience_analyzed'] = merged_df['preferredExperience_cleaned'].apply(analyze_foreign_words)
merged_df['preferredExperience_analyzed_all'] = merged_df['preferredExperience_cleaned'].apply(analyze_foreign_words_all)

print("형태소 분석 완료.")
print('-'*80)

형태소 분석 시작...
형태소 분석 완료.
--------------------------------------------------------------------------------


In [ ]:
# 7.5. 결과를 새로운 CSV 파일에 저장
output_columns = [
    'id',
    'description',
    'requirement',
    'preferredExperience',
    'description_requirement',  # 'description+requirement' -> 'description_requirement'으로 변경
    'combined_analyzed',  # 'description_requirement_analyzed' -> 'combined_analyzed'으로 변경
    'combined_analyzed_all',  # 'description_requirement_analyzed_all' -> 'combined_analyzed_all'으로 변경
    'preferredExperience_analyzed',
    'preferredExperience_analyzed_all'
]
# 필요한 열만 선택하여 저장 (존재하지 않는 열은 제외)
existing_output_columns = [col for col in output_columns if col in merged_df.columns]

# 출력 디렉토리가 존재하지 않으면 생성
output_dir = os.path.dirname(output_file)
if output_dir and not os.path.exists(output_dir):
    os.makedirs(output_dir)

# CSV 파일로 저장
merged_df[existing_output_columns].to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"형태소 분석 결과가 '{output_file}' 파일에 저장되었습니다.")

형태소 분석 결과가 '/content/[lower]analyzed_job_data_final.csv' 파일에 저장되었습니다.


# 형태소 분석 결과, 메타 DB로 항목화

In [ ]:
import json

# Convert the 'combined_analyzed' and 'preferredExperience_analyzed' columns to JSON format separately
df['combined_analyzed_json'] = df['combined_analyzed'].apply(lambda x: json.dumps(eval(x)) if isinstance(x, str) else json.dumps([]))
df['preferredExperience_analyzed_json'] = df['preferredExperience_analyzed'].apply(lambda x: json.dumps(eval(x)) if isinstance(x, str) else json.dumps([]))

# Save the result with the separate JSON columns
output_file_path_json = '/content/[lower]separate_json_columns_data.csv'
df[['id', 'combined_analyzed_json', 'preferredExperience_analyzed_json']].to_csv(output_file_path_json, index=False, encoding='utf-8-sig')

# Provide the link to download the file
output_file_path_json


'/content/[lower]separate_json_columns_data.csv'

# 코드너리 사전에 없는 기술들 추출한 결과

In [ ]:
import pandas as pd

# Define the categories provided (converted to lowercase)
languages = [
    'c++', 'c#', 'dart', 'elixir', 'groovy', 'java', 'javascript', 'lua', 'perl', 'php', 'python', 'r',
    'ruby', 'rust', 'scala', 'swift', 'typescript', 'rescript', 'go'
]
frameworks = [
    'angular', 'appium', 'armeria', 'backbonejs', 'codeigniter', 'dagger', 'dropwizard', 'echo', 'electron',
    'emberjs', 'expressjs', 'falcon', 'fastapi', 'fastify', 'fiber', 'flask', 'flink', 'gatsby', 'gin', 'grpc',
    'hadoop', 'jasmine', 'junit', 'kotest', 'ktor', 'laravel', 'meteor', 'mocha', 'mockito', 'nestjs', 'netty',
    'nextjs', 'nuxtjs', 'phoenix', 'ray', 'react native', 'reactorkit', 'relay', 'ribs', 'ruby on rails', 'sanic',
    'selenium', 'spark', 'spring', 'springboot', 'svelte', 'swagger', 'tailwind', 'thrift', 'vuejs', 'nodejs', 'koa'
]
libraries = [
    'alamofire', 'apollo', 'emotion', 'enzyme', 'exoplayer', 'glide', 'immer', 'jotai', 'keras', 'lottie', 'mobx',
    'moya', 'react context', 'reactivex', 'reactjs', 'react query', 'recoil', 'redux', 'rest-assured', 'retrofit',
    'sinon', 'snapkit', 'styled-components', 'tensorflow', 'testing library', 'vuex', 'zustand', 'pytorch', 'graphql'
]
tools = [
    'agit', 'airflow', 'ansible', 'arcus', 'arangodb', 'argo cd', 'asana', 'aws athena', 'aws auroradb', 'aws codebuild',
    'aws codedeploy', 'aws codepipeline', 'aws documentdb', 'aws dynamodb', 'aws kinesis', 'aws mariadb', 'aws redshift',
    'aws ses', 'aws sns', 'azure devops', 'bazel', 'bitbucket', 'bitrise', 'capistrano', 'cassandradb', 'celery', 'central dogma',
    'ceph', 'circle ci', 'clickhouse', 'cockroachdb', 'confluence', 'couchbase', 'cubrid', 'cucumber', 'cypress', 'discord', 'docker',
    'docusaurus', 'dooray', 'drone', 'druid', 'elasticsearch', 'fastlane', 'fluentd', 'flow', 'github', 'github action', 'gitlab',
    'go cd', 'google bigquery', 'google cloud build', 'google data studio', 'google firebase', 'grafana', 'greenplum', 'gulp', 'h2',
    'harbor', 'hbase', 'hazelcast', 'helm', 'hive', 'hugo', 'istio', 'jaeger', 'jandi', 'jenkins', 'jira', 'kafka', 'kakaotalk',
    'kakaowork', 'karma', 'karpenter', 'kibana', 'kube-bench', 'kubeflow', 'kubernetes', 'kudu', 'linkerd', 'liquibase', 'locust',
    'looker', 'memcached', 'metabase', 'microsoft-teams', 'mlflow', 'monday', 'mongodb', 'mssql', 'mysql', 'naver works', 'neo4j',
    'nexus', 'ngrinder', 'nifi', 'notion', 'openebs', 'oracledb', 'packer', 'playwright', 'postgresql', 'presto', 'prometheus',
    'rabbitmq', 'rancher', 'ranger', 'redash', 'redis', 'rocksdb', 'saltstack', 'slack', 'snowflake', 'solr', 'sonarqube',
    'storybook', 'superset', 'tableau', 'telegram', 'travis ci', 'trello', 'trino', 'tuist', 'unity', 'vault', 'zabbix', 'zeppelin',
    'playwright', 'puppeteer', 'opengl', 'zipkin', 'impala', 'influxdb', 'traefik'
]

# Load the uploaded file to check its contents
file_path = '/content/[lower]separate_json_columns_data.csv'
df = pd.read_csv(file_path)

# Function to filter out items that are in the categories
def filter_non_matching_items(tokens):
    if isinstance(tokens, str):
        tokens = eval(tokens)  # Convert string to list
    non_matching_items = [item for item in tokens if item.lower() not in
                          (languages + frameworks + libraries + tools)]
    return non_matching_items

# Apply filtering to both columns
df['combined_analyzed_non_matching'] = df['combined_analyzed_json'].apply(filter_non_matching_items)
df['preferredExperience_analyzed_non_matching'] = df['preferredExperience_analyzed_json'].apply(filter_non_matching_items)

# Save the filtered result to a new file
output_file_path_filtered = '/content/[lower_test1]filtered_non_matching_data.csv'
df[['id', 'combined_analyzed_non_matching', 'preferredExperience_analyzed_non_matching']].to_csv(output_file_path_filtered, index=False, encoding='utf-8-sig')

# Provide the link to download the file
output_file_path_filtered


'/content/[lower_test1]filtered_non_matching_data.csv'

# C를 제외한 1음절 단어 모두 제거한 결과

In [ ]:
# /content/[lower_test1]1음절_filtered_data.csv

In [ ]:
import pandas as pd
import json

# 파일 경로 및 컬럼 이름
file_path = '/content/[lower_test1]1음절_filtered_data.csv'
combined_col = 'combined_analyzed_filtered'
preferred_col = 'preferredExperience_analyzed_filtered'

# 대표값 리스트
categories = {
    "languages": [
        'c++', 'c#', 'dart', 'elixir', 'groovy', 'java', 'javascript', 'lua', 'perl', 'php', 'python', 'r',
        'ruby', 'rust', 'scala', 'swift', 'typescript', 'rescript', 'go'
    ],
    "frameworks": [
        'angular', 'appium', 'armeria', 'backbonejs', 'codeigniter', 'dagger', 'dropwizard', 'echo', 'electron',
        'emberjs', 'expressjs', 'falcon', 'fastapi', 'fastify', 'fiber', 'flask', 'flink', 'gatsby', 'gin', 'grpc',
        'hadoop', 'jasmine', 'junit', 'kotest', 'ktor', 'laravel', 'meteor', 'mocha', 'mockito', 'nestjs', 'netty',
        'nextjs', 'nuxtjs', 'phoenix', 'ray', 'react native', 'reactorkit', 'relay', 'ribs', 'ruby on rails', 'sanic',
        'selenium', 'spark', 'spring', 'springboot', 'svelte', 'swagger', 'tailwind', 'thrift', 'vuejs', 'nodejs', 'koa'
    ],
    "libraries": [
        'alamofire', 'apollo', 'emotion', 'enzyme', 'exoplayer', 'glide', 'immer', 'jotai', 'keras', 'lottie', 'mobx',
        'moya', 'react context', 'reactivex', 'reactjs', 'react query', 'recoil', 'redux', 'rest-assured', 'retrofit',
        'sinon', 'snapkit', 'styled-components', 'tensorflow', 'testing library', 'vuex', 'zustand', 'pytorch', 'graphql'
    ],
    "tools": [
        'agit', 'airflow', 'ansible', 'arcus', 'arangodb', 'argo cd', 'asana', 'aws athena', 'aws auroradb', 'aws codebuild',
        'aws codedeploy', 'aws codepipeline', 'aws documentdb', 'aws dynamodb', 'aws kinesis', 'aws mariadb', 'aws redshift',
        'aws ses', 'aws sns', 'azure devops', 'bazel', 'bitbucket', 'bitrise', 'capistrano', 'cassandradb', 'celery', 'central dogma',
        'ceph', 'circle ci', 'clickhouse', 'cockroachdb', 'confluence', 'couchbase', 'cubrid', 'cucumber', 'cypress', 'discord', 'docker',
        'docusaurus', 'dooray', 'drone', 'druid', 'elasticsearch', 'fastlane', 'fluentd', 'flow', 'github', 'github action', 'gitlab',
        'go cd', 'google bigquery', 'google cloud build', 'google data studio', 'google firebase', 'grafana', 'greenplum', 'gulp', 'h2',
        'harbor', 'hbase', 'hazelcast', 'helm', 'hive', 'hugo', 'istio', 'jaeger', 'jandi', 'jenkins', 'jira', 'kafka', 'kakaotalk',
        'kakaowork', 'karma', 'karpenter', 'kibana', 'kube-bench', 'kubeflow', 'kubernetes', 'kudu', 'linkerd', 'liquibase', 'locust',
        'looker', 'memcached', 'metabase', 'microsoft-teams', 'mlflow', 'monday', 'mongodb', 'mssql', 'mysql', 'naver works', 'neo4j',
        'nexus', 'ngrinder', 'nifi', 'notion', 'openebs', 'oracledb', 'packer', 'playwright', 'postgresql', 'presto', 'prometheus',
        'rabbitmq', 'rancher', 'ranger', 'redash', 'redis', 'rocksdb', 'saltstack', 'slack', 'snowflake', 'solr', 'sonarqube',
        'storybook', 'superset', 'tableau', 'telegram', 'travis ci', 'trello', 'trino', 'tuist', 'unity', 'vault', 'zabbix', 'zeppelin',
        'playwright', 'puppeteer', 'opengl', 'zipkin', 'impala', 'influxdb', 'traefik'
    ]
}

# 동의어 사전 생성 함수
def create_synonym_dict():
    df = pd.read_csv(file_path)
    synonym_dict = {category: {} for category in categories.keys()}

    for category, terms in categories.items():
        for term in terms:
            synonym_dict[category][term] = [term]  # 대표값 포함
            for col in [combined_col, preferred_col]:
                matches = df[col].dropna().apply(lambda x: term.replace(" ", "").lower() in str(x).replace(" ", "").lower())
                matched_terms = df[col][matches].str.lower().str.strip().unique().tolist()
                synonym_dict[category][term].extend([t for t in matched_terms if t != term])
            synonym_dict[category][term] = list(set(synonym_dict[category][term]))  # 중복 제거

    return synonym_dict

# 동의어 사전 생성
synonym_dict = create_synonym_dict()

# JSON 파일로 저장
with open('/content/[gayoon2_final]synonym_dict.json', 'w') as f:
    json.dump(synonym_dict, f, indent=2)

print("동의어 사전이 생성되었습니다!")


동의어 사전이 생성되었습니다!


In [ ]:
# 최종 : 내가 만든 동의어 사전은 final_synonym_corrected.json
# 현아님과 취합한 최종 동의어 사전
# updated_synonym_corrected.json


In [ ]:
import pandas as pd
import json

# Re-import the file after environment reset
file_path = '/content/[lower_test1]1음절_filtered_data.csv'

# Load the CSV file
df = pd.read_csv(file_path)

# Convert the DataFrame to JSON
json_data = df.to_json(orient='records', force_ascii=False, indent=2)

# Save the JSON file
output_path = '/content/[lower_test1]1음절_filtered_data.json'
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(json_data)

output_path


'/content/[lower_test1]1음절_filtered_data.json'

In [ ]:
import pandas as pd
import json

# Load the CSV file
file_path = '/content/[lower_test1]1음절_filtered_data.csv'
df = pd.read_csv(file_path)

# Columns to analyze
combined_col = 'combined_analyzed_filtered'
preferred_col = 'preferredExperience_analyzed_filtered'

# Define the categories
categories = {
    "languages": [
        'c++', 'c#', 'dart', 'elixir', 'groovy', 'java', 'javascript', 'lua', 'perl', 'php', 'python', 'r',
        'ruby', 'rust', 'scala', 'swift', 'typescript', 'rescript', 'go'
    ],
    "frameworks": [
        'angular', 'appium', 'armeria', 'backbonejs', 'codeigniter', 'dagger', 'dropwizard', 'echo', 'electron',
        'emberjs', 'expressjs', 'falcon', 'fastapi', 'fastify', 'fiber', 'flask', 'flink', 'gatsby', 'gin', 'grpc',
        'hadoop', 'jasmine', 'junit', 'kotest', 'ktor', 'laravel', 'meteor', 'mocha', 'mockito', 'nestjs', 'netty',
        'nextjs', 'nuxtjs', 'phoenix', 'ray', 'react native', 'reactorkit', 'relay', 'ribs', 'ruby on rails', 'sanic',
        'selenium', 'spark', 'spring', 'springboot', 'svelte', 'swagger', 'tailwind', 'thrift', 'vuejs', 'nodejs', 'koa'
    ],
    "libraries": [
        'alamofire', 'apollo', 'emotion', 'enzyme', 'exoplayer', 'glide', 'immer', 'jotai', 'keras', 'lottie', 'mobx',
        'moya', 'react context', 'reactivex', 'reactjs', 'react query', 'recoil', 'redux', 'rest-assured', 'retrofit',
        'sinon', 'snapkit', 'styled-components', 'tensorflow', 'testing library', 'vuex', 'zustand', 'pytorch', 'graphql'
    ],
    "tools": [
        'agit', 'airflow', 'ansible', 'arcus', 'arangodb', 'argo cd', 'asana', 'aws athena', 'aws auroradb', 'aws codebuild',
        'aws codedeploy', 'aws codepipeline', 'aws documentdb', 'aws dynamodb', 'aws kinesis', 'aws mariadb', 'aws redshift',
        'aws ses', 'aws sns', 'azure devops', 'bazel', 'bitbucket', 'bitrise', 'capistrano', 'cassandradb', 'celery', 'central dogma',
        'ceph', 'circle ci', 'clickhouse', 'cockroachdb', 'confluence', 'couchbase', 'cubrid', 'cucumber', 'cypress', 'discord', 'docker',
        'docusaurus', 'dooray', 'drone', 'druid', 'elasticsearch', 'fastlane', 'fluentd', 'flow', 'github', 'github action', 'gitlab',
        'go cd', 'google bigquery', 'google cloud build', 'google data studio', 'google firebase', 'grafana', 'greenplum', 'gulp', 'h2',
        'harbor', 'hbase', 'hazelcast', 'helm', 'hive', 'hugo', 'istio', 'jaeger', 'jandi', 'jenkins', 'jira', 'kafka', 'kakaotalk',
        'kakaowork', 'karma', 'karpenter', 'kibana', 'kube-bench', 'kubeflow', 'kubernetes', 'kudu', 'linkerd', 'liquibase', 'locust',
        'looker', 'memcached', 'metabase', 'microsoft-teams', 'mlflow', 'monday', 'mongodb', 'mssql', 'mysql', 'naver works', 'neo4j',
        'nexus', 'ngrinder', 'nifi', 'notion', 'openebs', 'oracledb', 'packer', 'playwright', 'postgresql', 'presto', 'prometheus',
        'rabbitmq', 'rancher', 'ranger', 'redash', 'redis', 'rocksdb', 'saltstack', 'slack', 'snowflake', 'solr', 'sonarqube',
        'storybook', 'superset', 'tableau', 'telegram', 'travis ci', 'trello', 'trino', 'tuist', 'unity', 'vault', 'zabbix', 'zeppelin',
        'playwright', 'puppeteer', 'opengl', 'zipkin', 'impala', 'influxdb', 'traefik'
    ]
}

# Initialize the synonym dictionary
synonym_dict = {category: {term: [] for term in terms} for category, terms in categories.items()}

# Iterate through the DataFrame and find matches for each category
for category, terms in categories.items():
    for term in terms:
        # Check for matches in combined_analyzed_filtered column
        combined_matches = df[combined_col].str.contains(term, case=False, na=False)
        synonym_dict[category][term].extend(df[combined_col][combined_matches].unique().tolist())

        # Check for matches in preferredExperience_analyzed_filtered column
        preferred_matches = df[preferred_col].str.contains(term, case=False, na=False)
        synonym_dict[category][term].extend(df[preferred_col][preferred_matches].unique().tolist())

        # Remove duplicates and keep only unique synonyms
        synonym_dict[category][term] = list(set(synonym_dict[category][term]))

# Save the synonym dictionary as a JSON file
output_path = '/content/[FINAL_GY]synonym_dictionary.json'
with open(output_path, 'w') as json_file:
    json.dump(synonym_dict, json_file, indent=4)

output_path


'/content/[FINAL_GY]synonym_dictionary.json'

In [ ]:
# 우리가 식별하지 못한 목록 (기존 값과 중복x)
categories = {
    "languages": ['python', 'java', 'javascript', 'c', 'c++', 'c#', 'r', 'ruby', 'swift', 'kotlin', 'typescript',
                  'go', 'php', 'perl', 'scala', 'bash', 'lua', 'matlab', 'rust'],
    "frameworks": ['django', 'flask', 'spring', 'springboot', 'express', 'react', 'angular', 'vue', 'nextjs', 'nuxtjs',
                   'fastapi', 'grpc', 'spark', 'hadoop', 'nestjs', 'svelte', 'emberjs', 'rails', 'fiber', 'koa'],
    "libraries": ['tensorflow', 'pytorch', 'scikit-learn', 'numpy', 'pandas', 'seaborn', 'matplotlib', 'keras',
                  'huggingface', 'xgboost', 'lightgbm', 'grpcio', 'opencv', 'beautifulsoup', 'sqlalchemy', 'requests',
                  'fastai', 'bokeh', 'plotly', 'transformers', 'pydantic'],
    "tools": ['docker', 'kubernetes', 'ansible', 'terraform', 'jenkins', 'circleci', 'airflow', 'grafana', 'prometheus',
              'zabbix', 'elasticsearch', 'mongodb', 'redis', 'postgresql', 'mysql', 'neo4j', 'clickhouse', 'rabbitmq',
              'kafka', 'snowflake', 'superset', 'tableau', 'metabase', 'apache spark', 'hive', 'hbase', 'cassandra',
              'helm', 'istio', 'jaeger', 'linkerd', 'dask', 'bigquery', 'aws', 'azure', 'gcp', 'openstack']
}

# 프레임워크 - 제공된 틀만 넣어서 실행
# 라이브러리 - 자유롭게 사용할 함수를 조합해서 사용

# 우리가 식별하지 못한 목록(중복제거)
categories = {
    "languages": ['bash', 'matlab', 'c', 'kotlin'],
    "frameworks": ['react', 'express', 'django', 'rails', 'vue'],
    "libraries": ['sqlalchemy', 'transformers', 'matplotlib', 'opencv', 'huggingface', 'lightgbm', 'pandas', 'pydantic',
                  'beautifulsoup', 'plotly', 'xgboost', 'grpcio', 'numpy', 'scikit-learn', 'bokeh', 'seaborn',
                  'requests', 'fastai'],
    "tools": ['bigquery', 'terraform', 'azure', 'gcp', 'openstack', 'cassandra', 'apache spark', 'dask', 'aws',
              'circleci']
}


# 우리가 가진 목록
categories = {
    "languages": ['c++', 'c#', 'dart', 'elixir', 'groovy', 'java', 'javascript', 'lua', 'perl', 'php', 'python', 'r',
                  'ruby', 'rust', 'scala', 'swift', 'typescript', 'rescript', 'go'],
    "frameworks": ['angular', 'appium', 'armeria', 'backbonejs', 'codeigniter', 'dagger', 'dropwizard', 'echo',
                   'electron', 'emberjs', 'expressjs', 'falcon', 'fastapi', 'fastify', 'fiber', 'flask', 'flink',
                   'gatsby', 'gin', 'grpc', 'hadoop', 'jasmine', 'junit', 'kotest', 'ktor', 'laravel', 'meteor',
                   'mocha', 'mockito', 'nestjs', 'netty', 'nextjs', 'nuxtjs', 'phoenix', 'ray', 'react native',
                   'reactorkit', 'relay', 'ribs', 'ruby on rails', 'sanic', 'selenium', 'spark', 'spring',
                   'springboot', 'svelte', 'swagger', 'tailwind', 'thrift', 'vuejs', 'nodejs', 'koa'],
    "libraries": ['alamofire', 'apollo', 'emotion', 'enzyme', 'exoplayer', 'glide', 'immer', 'jotai', 'keras', 'lottie',
                  'mobx', 'moya', 'react context', 'reactivex', 'reactjs', 'react query', 'recoil', 'redux',
                  'rest-assured', 'retrofit', 'sinon', 'snapkit', 'styled-components', 'tensorflow', 'testing library',
                  'vuex', 'zustand', 'pytorch', 'graphql'],
    "tools": ['agit', 'airflow', 'ansible', 'arcus', 'arangodb', 'argo cd', 'asana', 'aws athena', 'aws auroradb',
              'aws codebuild', 'aws codedeploy', 'aws codepipeline', 'aws documentdb', 'aws dynamodb', 'aws kinesis',
              'aws mariadb', 'aws redshift', 'aws ses', 'aws sns', 'azure devops', 'bazel', 'bitbucket', 'bitrise',
              'capistrano', 'cassandradb', 'celery', 'central dogma', 'ceph', 'circle ci', 'clickhouse', 'cockroachdb',
              'confluence', 'couchbase', 'cubrid', 'cucumber', 'cypress', 'discord', 'docker', 'docusaurus', 'dooray',
              'drone', 'druid', 'elasticsearch', 'fastlane', 'fluentd', 'flow', 'github', 'github action', 'gitlab',
              'go cd', 'google bigquery', 'google cloud build', 'google data studio', 'google firebase', 'grafana',
              'greenplum', 'gulp', 'h2', 'harbor', 'hbase', 'hazelcast', 'helm', 'hive', 'hugo', 'istio', 'jaeger',
              'jandi', 'jenkins', 'jira', 'kafka', 'kakaotalk', 'kakaowork', 'karma', 'karpenter', 'kibana',
              'kube-bench', 'kubeflow', 'kubernetes', 'kudu', 'linkerd', 'liquibase', 'locust', 'looker', 'memcached',
              'metabase', 'microsoft-teams', 'mlflow', 'monday', 'mongodb', 'mssql', 'mysql', 'naver works', 'neo4j',
              'nexus', 'ngrinder', 'nifi', 'notion', 'openebs', 'oracledb', 'packer', 'playwright', 'postgresql',
              'presto', 'prometheus', 'rabbitmq', 'rancher', 'ranger', 'redash', 'redis', 'rocksdb', 'saltstack',
              'slack', 'snowflake', 'solr', 'sonarqube', 'storybook', 'superset', 'tableau', 'telegram', 'travis ci',
              'trello', 'trino', 'tuist', 'unity', 'vault', 'zabbix', 'zeppelin', 'playwright', 'puppeteer', 'opengl',
              'zipkin', 'impala', 'influxdb', 'traefik']
}

In [ ]:
import pandas as pd
import json

# Load CSV file
file_path = '/content/[lower_test1]1음절_filtered_data.csv'
df = pd.read_csv(file_path)

# Define columns to search in
combined_col = 'combined_analyzed_filtered'
preferred_col = 'preferredExperience_analyzed_filtered'

# Define representative categories
categories = {
    "languages": ['c++', 'c#', 'dart', 'elixir', 'groovy', 'java', 'javascript', 'lua', 'perl', 'php', 'python', 'r',
                  'ruby', 'rust', 'scala', 'swift', 'typescript', 'rescript', 'go'],
    "frameworks": ['angular', 'appium', 'armeria', 'backbonejs', 'codeigniter', 'dagger', 'dropwizard', 'echo',
                   'electron', 'emberjs', 'expressjs', 'falcon', 'fastapi', 'fastify', 'fiber', 'flask', 'flink',
                   'gatsby', 'gin', 'grpc', 'hadoop', 'jasmine', 'junit', 'kotest', 'ktor', 'laravel', 'meteor',
                   'mocha', 'mockito', 'nestjs', 'netty', 'nextjs', 'nuxtjs', 'phoenix', 'ray', 'react native',
                   'reactorkit', 'relay', 'ribs', 'ruby on rails', 'sanic', 'selenium', 'spark', 'spring',
                   'springboot', 'svelte', 'swagger', 'tailwind', 'thrift', 'vuejs', 'nodejs', 'koa'],
    "libraries": ['alamofire', 'apollo', 'emotion', 'enzyme', 'exoplayer', 'glide', 'immer', 'jotai', 'keras', 'lottie',
                  'mobx', 'moya', 'react context', 'reactivex', 'reactjs', 'react query', 'recoil', 'redux',
                  'rest-assured', 'retrofit', 'sinon', 'snapkit', 'styled-components', 'tensorflow', 'testing library',
                  'vuex', 'zustand', 'pytorch', 'graphql'],
    "tools": ['agit', 'airflow', 'ansible', 'arcus', 'arangodb', 'argo cd', 'asana', 'aws athena', 'aws auroradb',
              'aws codebuild', 'aws codedeploy', 'aws codepipeline', 'aws documentdb', 'aws dynamodb', 'aws kinesis',
              'aws mariadb', 'aws redshift', 'aws ses', 'aws sns', 'azure devops', 'bazel', 'bitbucket', 'bitrise',
              'capistrano', 'cassandradb', 'celery', 'central dogma', 'ceph', 'circle ci', 'clickhouse', 'cockroachdb',
              'confluence', 'couchbase', 'cubrid', 'cucumber', 'cypress', 'discord', 'docker', 'docusaurus', 'dooray',
              'drone', 'druid', 'elasticsearch', 'fastlane', 'fluentd', 'flow', 'github', 'github action', 'gitlab',
              'go cd', 'google bigquery', 'google cloud build', 'google data studio', 'google firebase', 'grafana',
              'greenplum', 'gulp', 'h2', 'harbor', 'hbase', 'hazelcast', 'helm', 'hive', 'hugo', 'istio', 'jaeger',
              'jandi', 'jenkins', 'jira', 'kafka', 'kakaotalk', 'kakaowork', 'karma', 'karpenter', 'kibana',
              'kube-bench', 'kubeflow', 'kubernetes', 'kudu', 'linkerd', 'liquibase', 'locust', 'looker', 'memcached',
              'metabase', 'microsoft-teams', 'mlflow', 'monday', 'mongodb', 'mssql', 'mysql', 'naver works', 'neo4j',
              'nexus', 'ngrinder', 'nifi', 'notion', 'openebs', 'oracledb', 'packer', 'playwright', 'postgresql',
              'presto', 'prometheus', 'rabbitmq', 'rancher', 'ranger', 'redash', 'redis', 'rocksdb', 'saltstack',
              'slack', 'snowflake', 'solr', 'sonarqube', 'storybook', 'superset', 'tableau', 'telegram', 'travis ci',
              'trello', 'trino', 'tuist', 'unity', 'vault', 'zabbix', 'zeppelin', 'playwright', 'puppeteer', 'opengl',
              'zipkin', 'impala', 'influxdb', 'traefik']
}

# Initialize final result dictionary
final_synonyms = {key: {term: [] for term in terms} for key, terms in categories.items()}

# Process the data
for key, terms in categories.items():
    for term in terms:
        for col in [combined_col, preferred_col]:
            matches = df[col].dropna().str.contains(term, case=False, na=False)
            similar_items = df[col][matches].unique()
            final_synonyms[key][term].extend(item.lower() for item in similar_items if item.lower() != term)

# Remove duplicates
for key in final_synonyms:
    for term in final_synonyms[key]:
        final_synonyms[key][term] = list(set(final_synonyms[key][term]))

# Save to JSON
output_path = '/content/[0123_gy]final_synonym_corrected.json'
with open(output_path, 'w') as json_file:
    json.dump(final_synonyms, json_file, indent=4)

print(f"Synonym dictionary saved to {output_path}")


Synonym dictionary saved to /content/[0123_gy]final_synonym_corrected.json


In [ ]:
import pandas as pd

# 파일 경로 설정
file_path = '/content/[lower_test1]1음절_filtered_data.csv'
output_path = '/content/[lower_test1]중복제거_filtered_data.csv'

# CSV 파일 읽기
df = pd.read_csv(file_path)

# 중복 제거
df['preferredExperience_analyzed_filtered'] = df['preferredExperience_analyzed_filtered'].drop_duplicates().reset_index(drop=True)
df['combined_analyzed_filtered'] = df['combined_analyzed_filtered'].drop_duplicates().reset_index(drop=True)

# 파일 저장
df.to_csv(output_path, index=False)

print(f"중복 제거된 파일이 저장되었습니다: {output_path}")


중복 제거된 파일이 저장되었습니다: /content/[lower_test1]중복제거_filtered_data.csv


In [ ]:
# 파일 경로 설정
file_path = '/content/[lower_test1]1음절_filtered_data.csv'
output_path = '/content/[lower_test1]중복제거2_filtered_data.csv'


In [ ]:
import pandas as pd

# 열 단위로 중복 제거 함수
def remove_duplicates_per_column(df, columns):
    """
    DataFrame의 특정 열에서 중복된 값을 제거합니다.
    각 열의 중복을 독립적으로 처리합니다.
    :param df: pandas DataFrame
    :param columns: 중복 제거를 수행할 열 이름 목록
    :return: 중복 제거된 DataFrame
    """
    for column in columns:
        # 각 열에서 중복 제거
        df[column] = df[column].apply(lambda x: ','.join(set(str(x).split(','))) if pd.notnull(x) else x)
    return df

# 파일 경로 설정
file_path = '/content/[lower_test1]1음절_filtered_data.csv'
output_path = '/content/[lower_test1]_deduplicated_filtered_data.csv'

# CSV 파일 읽기
df = pd.read_csv(file_path)

# 중복 제거 대상 열
columns_to_deduplicate = ['preferredExperience_analyzed_filtered', 'combined_analyzed_filtered']

# 열 단위 중복 제거
df = remove_duplicates_per_column(df, columns_to_deduplicate)

# 결과 저장
df.to_csv(output_path, index=False)

print(f"중복 제거된 파일이 저장되었습니다: {output_path}")


중복 제거된 파일이 저장되었습니다: /content/[lower_test1]_deduplicated_filtered_data.csv


# 동의어 사전(기술 사전)에 2개의 칼럼에 값들이 있는지 확인하고, 있다면 메타DB 항목으로 넣어주기(칼럼)


In [ ]:
*# Google Colab에서 실행할 Python 코드
import pandas as pd
import json

# JSON 데이터 경로 및 CSV 데이터 경로
json_file_path = '/content/updated_synonym_corrected_final.json'
csv_file_path = '//content/[lower_test1]1음절_filtered_data.csv'

# JSON 데이터 로드
with open(json_file_path, 'r') as file:
    synonym_data = json.load(file)

# CSV 데이터 로드
csv_data = pd.read_csv(csv_file_path)

# 분석할 컬럼
combined_column = 'combined_analyzed_filtered'
preferred_column = 'preferredExperience_analyzed_filtered'

# JSON 데이터 기반으로 카테고리를 찾는 함수
def find_category(term, categories):
    for category, items in categories.items():
        for key, synonyms in items.items():
            if term.lower() in [syn.lower() for syn in synonyms]:
                return category
    return None

# 새로운 컬럼 생성
csv_data['combined_category'] = csv_data[combined_column].apply(lambda x: find_category(str(x), synonym_data))
csv_data['preferred_category'] = csv_data[preferred_column].apply(lambda x: find_category(str(x), synonym_data))

# 결과를 새 CSV 파일로 저장
output_file_path = '/content/updated_filtered_with_categories.csv'
csv_data.to_csv(output_file_path, index=False)

print(f"파일이 저장되었습니다: {output_file_path}")


# 아래는 무시 : 분석 속도 향상을 위한 청크 크기 설정

In [ ]:

# 5. 청크 크기 설정
chunksize = 10000  # 필요에 따라 조정

# 6. 출력 파일 초기화 (헤더 포함)
if os.path.exists(output_file):
    os.remove(output_file)

# 7. CSV 파일 청크 단위로 읽기 및 처리
for chunk_number, chunk in enumerate(pd.read_csv(merged_df_file, chunksize=chunksize), start=1):
    print(f"청크 {chunk_number} 처리 시작...")

    # 7.1. 필요한 열이 존재하는지 확인하고, 없으면 빈 문자열로 채움
    required_columns = ['id', 'description', 'requirement', 'preferredExperience']
    for col in required_columns:
        if col not in chunk.columns:
            chunk[col] = ''

    # 7.2. 'description'과 'requirement'를 합쳐 새로운 열 생성
    chunk['description+requirement'] = (
        chunk['description'].fillna('') + ' ' +
        chunk['requirement'].fillna('')
    )

    # 7.3. 'preferredExperience'는 별도로 새로운 열 생성
    chunk['preferredExperience_cleaned'] = chunk['preferredExperience'].fillna('')

    # 7.4. 형태소 분석 적용
    chunk['description_requirement_analyzed'] = chunk['description+requirement'].apply(analyze_text_with_custom_words)
    chunk['description_requirement_analyzed_all'] = chunk['description+requirement'].apply(analyze_text_all_with_custom_words)
    chunk['preferredExperience_analyzed'] = chunk['preferredExperience_cleaned'].apply(analyze_text_with_custom_words)
    chunk['preferredExperience_analyzed_all'] = chunk['preferredExperience_cleaned'].apply(analyze_text_all_with_custom_words)

    print("형태소 분석 완료.")
    print('-'*80)
